In [1]:
import os 
import io
import openai
from transformers.models.imagegpt.modeling_imagegpt import IMAGEGPT_INPUTS_DOCSTRING
from transformers import GPT2Tokenizer
import pandas as pd 
from pandas.io import json
from numpy import nan
import time
import csv
import shutil
import datetime
import pytz
import re
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = "your_api_key"
#openai.api_key  = os.getenv('OPENAI_API_KEY')
model_engine = "your_selected_model"
prompt_name = 'your_prompt'

In [3]:
def user_input():
    file_path = 'Prompt/'+ prompt_name +'.txt'

    with open(file_path, 'r') as file:
        user_input = file.read()

    dataset = "your_dataset"
    temperature = 0.0
    query = user_input

    # Printing the extracted values
    print(dataset)
    print(temperature)
    print(query)
    
    return dataset, temperature,  query

dataset, temperature, query = user_input()
Implementation_base_path_input = 'your_dataset_path'
print(Implementation_base_path_input)

AdjuvareDB104
0.0
Task: Extract vaccine adjuvant names from the provided clinical trial data. Each line of the input includes a unique identifier (NCT Number) and a study summary of the clinical trial, separated by a tab. Multiple interventions are separared by "|". Your task is to identify any mention of adjuvants in the data and pair it with the corresponding NCT Number. The output format and examples are provided below.

```
Output Format:Produce a TSV (tab-separated values) output file with the two following columns:
* NCT Number: The unique identifier for each trial.
* Adjuvant Name: The name of the adjuvant mentioned in the provided clinical trial information.
If multiple adjuvants are mentioned, list each adjuvant in a separate output line. The output should not contain any blank columns. The output should include a line with 'Done' at the end. 
```

```
Input example:
NCT Number	Study Summary
NCT01479244	Study Title: Efficacy and Safety Study of NeuVax (Nelipepimut-S or E75) Va

In [4]:
### Preprocessing  ###
# List all .txt files in the directory
filenames = [f for f in os.listdir(Implementation_base_path_input) if f.endswith('.txt')]

# Process each file
for filename in filenames:
    full_path = os.path.join(Implementation_base_path_input, filename)
    with open(full_path, 'r') as file:
        data = file.read()

    # Clean the data
    cleaned_value = data.replace("|", " | ")

    # Write the cleaned data back to the file
    with open(full_path, 'w') as file:
        file.write(cleaned_value)

'Implementation_base_path_input = \'Dataset/\'+dataset+\'_Standard/10_folds_preprocessed_with_intervention_merged_columns\'\n\n\n# List all .txt files in the directory\nfilenames = [f for f in os.listdir(Implementation_base_path_input) if f.endswith(\'.txt\')]\n\n# Process each file\nfor filename in filenames:\n    full_path = os.path.join(Implementation_base_path_input, filename)\n    with open(full_path, \'r\') as file:\n        data = file.read()\n\n    # Clean the data\n    cleaned_value = data.replace("|", " | ")\n\n    # Write the cleaned data back to the file\n    with open(full_path, \'w\') as file:\n        file.write(cleaned_value)\n'

In [5]:
#count_input_tokens(Implementation_base_path_input)

from transformers import GPT2Tokenizer
from math import ceil

def count_input_tokens_no_use(folder_path):
  print(folder_path)
  # Initialize GPT2 tokenizer
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  # Get list of file names in folder
  file_names = os.listdir(folder_path)
  # List to store file names with more than 1500 tokens
  large_token_files = []
  x = 0
  y = 0
  parts = 0
  # Loop through each file and calculate number of tokens
  for file_name in file_names:
      # Check if file is a text file
      if file_name.endswith(".csv"):
          # Read file contents
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as f:
              file_contents = f.read()
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as fp:
              num_input_line = len(fp.readlines())
          # Calculate number of tokens
          num_tokens = len(tokenizer.encode(file_contents))
          num_output_tokens  = num_input_line*100
          total_token = num_tokens + num_output_tokens
          if total_token > 16000:
            print(f"{file_name}: {total_token} tokens \t parts needed: {ceil(num_output_tokens/4000)}")
            x= x+1
            parts = parts + ceil(num_output_tokens/4000)
            large_token_files.append(file_name)
          else:
            y = y+1
            parts = parts +1
        
  print(x)
  print(y)
  print(parts)
  return large_token_files
              
            
large_token_files = count_input_tokens_no_use(Implementation_base_path_input)
print(large_token_files)
#### Split the files with large tokens to meet the context window of your desired model


Dataset/AdjuvareDB104_Standard/10_folds_preprocessed_with_intervention_merged_columns
0
0
0
[]


In [7]:
import time
import datetime
import pytz
import openai

def get_completion(BACKOFF_OCCURRED_, Sentences, max_tokens, temperature, query=query, model=model_engine):
    prompt = f"""
    {query}
    {Sentences}
    ```
    """
    #print(prompt)
    messages = [{"role": "user", "content": prompt}]
    
    tries = 0
    
    while True:
        try:
            start_time = time.time()
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            call_time = time.time()
            time_f = call_time - start_time
            break
        except (openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.APIError)  as e:
            tries += 1
            max_backoff = 60  # Example: maximum of 60 seconds
            backoff_time =  min(5 + 5*(tries ** 2), max_backoff)
            #BACKOFF_OCCURRED_ = True # Reset the backoff 
            with BACKOFF_OCCURRED_.get_lock():
                BACKOFF_OCCURRED_.value = True
            #print(f"Rate limit exceeded. Retrying in {backoff_time} seconds...")
            #print("############################\nNumber of Failure:", tries)
            #print(f"###########################\nRate limit exceeded. Retrying in {backoff_time} seconds...")
            
            time.sleep(backoff_time)
            with BACKOFF_OCCURRED_.get_lock():
                BACKOFF_OCCURRED_.value = False
            print("Backoff Released\n")
    
    message = response['choices'][0]['message']['content']
    output_token = response['usage']['completion_tokens']
    input_token = response['usage']['prompt_tokens']
    
    now_utc = datetime.datetime.now(pytz.utc)
    timezone = pytz.timezone("US/Central")
    now_eastern = now_utc.astimezone(timezone)
    time_stamp = str(now_eastern)
    time.sleep(1)
    
    return message, input_token, output_token, time_f, time_stamp


In [8]:
import os
import itertools
import random
import time

def call_ChatGPT(BACKOFF_OCCURRED_, Implementation_base_path_input, Implementation_base_path_output, Run, temperature, query, input_file_name):
    #print("Maximum number of Token: ", total_max_tokens)
    #print("Temperature: ", temperature)
    #print("Query:", query)
    #print("Dataset: ", dataset)
    #print ("Number of folds:", D_folds)
    
    #r = D_folds+1
    
    RETRY_COUNT = 5
    ########################################################################
    #if BACKOFF_OCCURRED_:
    while BACKOFF_OCCURRED_.value:
        print("\nBackoff occurred! Pausing all threads for a set duration...")
        sleep_time = random.randint(1, 5)
        time.sleep(sleep_time) # Adjust this duration as per your requirements.
        #BACKOFF_OCCURRED_ = False
        #BACKOFF_OCCURRED_.value = False
            
    #########################################################################    
            
    for attempt in range(RETRY_COUNT):
        try:

        #for filename in input_file_names:
            with open(os.path.join(Implementation_base_path_input, input_file_name)) as f:
                Sentences = f.read()

            num_lines = sum(1 for line in open(os.path.join(Implementation_base_path_input, input_file_name)))

            total_max_tokens = num_lines*100+50
            print(total_max_tokens)

            message, input_token, output_token, time_f, time_stamp = get_completion(
                BACKOFF_OCCURRED_, Sentences=Sentences, max_tokens=total_max_tokens, temperature=temperature
            )

            last_line = message.strip().split('\n')[-1]

            if "Done" in last_line:
                print(f"File = {input_file_name}, Temperature = {temperature}, Complete")
            else:
                print(f"File = {input_file_name}, Temperature = {temperature}, Possibly Incomplete")

            file_name = os.path.basename(input_file_name)
            base_name = os.path.splitext(file_name)[0] + ".txt"

            output_file = os.path.join(Implementation_base_path_output, base_name)

            with open(output_file, "w") as f:
                print(message, file=f)
            break
                
        except Exception as e:  # Catch general exceptions. Be specific if you know which exceptions to expect
            print(f"Error occurred: {e}. Retrying {attempt+1}/{RETRY_COUNT}. input_file_name: {input_file_name}")
            time.sleep(1)  # Wait for 5 seconds before retrying. Adjust as necessary.

        else:  # This block will be executed if the for loop completed without 'break', i.e., if all attempts failed.
            print(f"All {RETRY_COUNT} retries failed for fold {current_folds} at Run {Run}. input_file_name: {input_file_name}")
        #time.sleep(1)


In [9]:
import os

def output_path(Run_no, dataset, temperature):
    base = "Output/"+prompt_name+"/GPT_"+model_engine+"/" + dataset + "_T"+ str(temperature)+ '/'
    extension_path = dataset + "_T" + str(temperature) + "_Run" + str(Run_no) + '/'
    Implementation_base_path_output = os.path.join(base, extension_path)

    # Use exist_ok=True to avoid FileExistsError in a multiprocessing context
    os.makedirs(Implementation_base_path_output, exist_ok=True)
        
    return Implementation_base_path_output


In [10]:
input_file_names = []

for file_name in os.listdir(Implementation_base_path_input):
    
    if os.path.isfile(os.path.join(Implementation_base_path_input, file_name)):
        input_file_names.append(file_name)

file_count = len(input_file_names)

print("Number of files in the folder:", file_count)
#print("File names:", input_file_names)

Number of files in the folder: 10


In [13]:
import multiprocessing

# Setting up the shared variable
BACKOFF_OCCURRED_ = multiprocessing.Value('b', False)  # 'b' denotes a boolean

def process_file(input_file_name, Run, dataset, temperature, query):
    
    global BACKOFF_OCCURRED_
    
    Implementation_base_path_output = output_path(Run, dataset,temperature)
    with open(os.path.join(Implementation_base_path_input, input_file_name)) as f:
        Sentences = f.read()
        
    if os.path.exists(os.path.join(Implementation_base_path_input, input_file_name)):
        with open(os.path.join(Implementation_base_path_input, input_file_name)) as f:
            Sentences = f.read()
        call_ChatGPT(BACKOFF_OCCURRED_, Implementation_base_path_input, Implementation_base_path_output, Run, temperature, query, input_file_name)
    else:
        print(f"File not found: {os.path.join(Implementation_base_path_input, input_file_name)}")

def worker_wrapper(args):
    return process_file(*args)

if __name__ == "__main__":
    # Setup your variables: Run, dataset, temperature, query, input_file_names
    number_of_workers = 10  # Number of worker processes
    for Run in range(1, 4):
        # Arguments to pass to process_file for each input_file_name
        args_list = [(file_name, Run, dataset, temperature, query) for file_name in input_file_names]

        # Create a pool of worker processes
        with multiprocessing.Pool(processes=number_of_workers) as pool:
            # map function blocks until all results are ready
            pool.map(worker_wrapper, args_list)
        


Temperature: Temperature: Temperature: Temperature: Temperature: Temperature: Temperature: Temperature: Temperature: Temperature:          0.0 0.00.00.00.00.00.00.00.00.0









Query:Query:Query:Query:Query:Query:Query:Query:Query:  Query:      Task: Extract vaccine adjuvant names from the provided clinical trial data. Each line of the input includes a unique identifier (NCT Number) and a study summary of the clinical trial, separated by a tab. Multiple interventions are separared by "|". Your task is to identify any mention of adjuvants in the data and pair it with the corresponding NCT Number. The output format and examples are provided below.

```
Output Format:Produce a TSV (tab-separated values) output file with the two following columns:
* NCT Number: The unique identifier for each trial.
* Adjuvant Name: The name of the adjuvant mentioned in the provided clinical trial information.
If multiple adjuvants are mentioned, list each adjuvant in a separate output line. The output s

### End of Code  ###